# Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt

from Assets.Loading_Datasets import *

np.random.seed(1)

# Datasets

Arguments:

    train_lable:    It declares which fruit this data belongs
    train_value:    It has value of every pixle

In [2]:
train_data = np.array(train_set, dtype=object)

#train_data

m, n = train_data.shape

np.random.shuffle(train_data)

data_train1 = train_data[0:200].T
train_value1 = data_train1[0]
train_lable1 = data_train1[1:n]

data_train2 = train_data[200:m].T
train_value2 = data_train2[0]
train_lable2 = data_train2[1:n]

train_value1

# Initialization
layer_dims: python array (list) containing the dimensions of each layer in our network

parameters: python dictionary containing our parameters "W1", "b1", ..., "WL", "bL":

                    Wl: weight matrix of shape (layer_dims[l], layer_dims[l-1])
                    bl: bias vector of shape (layer_dims[l], 1)


procedure:

we assign a random number for every weight and a 0 for all biases 

In [3]:
def initialize_parameters_deep(layer_dims):
   
    np.random.seed(3)
    parameters = {}
    L = len(layer_dims)            # number of layers in the network

    for l in range(1, L):
        
        parameters['W' + str(l)] = np.random.randn(layer_dims[l],layer_dims[l-1]) * 0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
        

        assert(parameters['W' + str(l)].shape == (layer_dims[l], layer_dims[l-1]))
        assert(parameters['b' + str(l)].shape == (layer_dims[l], 1))

    return parameters

# Network Shape

Our network will have 4 layers including output layer.

In the input layer, we have 102 items followed by 2 hidden layers with 150 and 60 items.

Also, in the output layer, we have for items which are: apple, lemon, mango and raspberry, respectively.

In [4]:
parameters = initialize_parameters_deep([102,150,60,4])
# print("W1 = " + str(parameters["W1"]))
# print("b1 = " + str(parameters["b1"]))
# print("W2 = " + str(parameters["W2"]))
# print("b2 = " + str(parameters["b2"]))
# print("W3 = " + str(parameters["W3"]))
# print("b3 = " + str(parameters["b3"]))

# Activation function

**Sigmoid:**
    
    Arguments:
    Z:       numpy array of any shape
    
    Returns:
    A:       output of sigmoid(z), same shape as Z
    cache:   returns Z as well, useful during backpropagation

**Relu:**

    Arguments:
    Z:       Output of the linear layer, of any shape
    Returns:
    A:       Post-activation parameter, of the same shape as Z
    cache:   storing "A" for computing the backward pass efficiently

In [5]:
def sigmoid(Z):
    
    A = 1/(1+np.exp(-Z))
    cache = Z
    
    return A, cache

In [6]:
def relu(Z):
    
    A = np.maximum(0,Z)
    
    assert(A.shape == Z.shape)
    
    cache = Z 
    return A, cache

# Forward-Propagation

Implement the **linear part** of a layer's forward propagation.

    Arguments:

    A:      activations from previous layer (or input data): (size of previous layer, number of examples)
    W:      weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b:      bias vector, numpy array of shape (size of the current layer, 1)

    Returns:

    Z:      the input of the activation function, also called pre-activation parameter.
    cache:  storing "A", "W" and "b" for computing the backward pass efficiently.

In [7]:
def linear_forward(A, W, b):
    
    Z = np.dot(W,A) + b
    
    # assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

# Activate the linear part of Forward-Propagation

Implement the forward propagation for the LINEAR->ACTIVATION part

    Arguments:

    A_prev:     activations from previous layer (or input data): (size of previous layer, number of examples)
    W:          weights matrix: numpy array of shape (size of current layer, size of previous layer)
    b:          bias vector, numpy array of shape (size of the current layer, 1)
    activation: the activation to be used in this layer, stored as a text string: "sigmoid" or "relu"

    Returns:

    A:          the output of the activation function, also called the post-activation value 
    cache:      storing "linear_cache" and "activation_cache" for computing the backward pass efficiently

In [11]:
def linear_activation_forward(A_prev, W, b, activation):
    
    if activation == "sigmoid":
    
        Z, linear_cache = linear_forward(A_prev, W, b) 
        A, activation_cache = sigmoid(Z) 
    
    elif activation == "relu":
    
        Z, linear_cache = linear_forward(A_prev, W, b)
        A, activation_cache = relu(Z) 
    
    # assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)

    return A, cache

# Forward-Propagation for all layers
    
**Arguments:**

    X:           data, numpy array of shape (input size, number of examples)
    parameters:  output of initialize_parameters_deep()
    
**Returns:**

    A:          last post-activation value
    caches:      list of caches containing:
                    every cache of activating linear part of forward-propagation (there are L-1 of them, indexed from 0 to L-1)

In [12]:
def L_model_forward(X, parameters):

    caches = []
    A = X
    L = len(parameters) // 2                  # number of layers in the neural network
    
    for l in range(1, L+1):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)], parameters['b' + str(l)], "sigmoid")
        caches.append(cache)
    
    # assert(A.shape == (1,X.shape[1]))
            
    return A, caches

# Cost function


We can use 2 ways to calculate the cost of network:

                                                SSE
                                                CE

**SSE**

In [13]:
def compute_SSE(value, lable):
    sse = 0
    m = label[0].shape
    for j in range(4):
        sse += (label[j] - value[j])^2
    
    return sse

**Cross**-**Entropy**

Arguments:

        AL:     probability vector corresponding to our label predictions, shape (1, number of examples)
        Y:      true "label" vector

Returns:

        cost:   cross-entropy cost

In [14]:
def compute_cost(AL, Y):
    
    m = Y.shape[1]

    # Compute loss from aL and y.
    cost = (-1/m) * (np.dot(Y, np.log(AL).T) + np.dot((1-Y), np.log(1-AL).T))
    
    # cost = np.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    # assert(cost.shape == ())
    
    return cost

# Back-Propagation

We need to calculate Gradient of the cost respect to the parameters. then times it to a learning factor and finally upgrade the parameters.

We will use 2 ways:
                    one way is that we use some loops to iterate all over the network
                    another way is that we use matrix operations 

# Using Loops

In [36]:
def calculate_gradient(value, lable):
    for i in range(value.size):
        A, caches = L_model_forward(value[i], parameters)
        sse = compute_SSE(A.T, lable[i])

# Vectorization

Using matrix operations

# Activation function Gradient

Implement the backward propagation for a single **SIGMOID** unit.

    Arguments:

        dA:      post-activation gradient, of any shape
        cache:   Z where we store for computing backward propagation efficiently

    Returns:
    
        dZ:      Gradient of the cost with respect to Z

In [15]:
def sigmoid_backward(dA, cache):
    
    Z = cache
    
    s = 1/(1+np.exp(-Z))
    dZ = dA * s * (1-s)
    
    assert (dZ.shape == Z.shape)
    
    return dZ

In [8]:
def relu_backward(dA, cache):
    """
    Implement the backward propagation for a single RELU unit.
    Arguments:
    dA -- post-activation gradient, of any shape
    cache -- 'Z' where we store for computing backward propagation efficiently
    Returns:
    dZ -- Gradient of the cost with respect to Z
    """
    
    Z = cache
    dZ = np.array(dA, copy=True) # just converting dz to a correct object.
    
    # When z <= 0, you should set dz to 0 as well. 
    dZ[Z <= 0] = 0
    
    assert (dZ.shape == Z.shape)
    
    return dZ

# Gradient of parameters for a singl layer

Here **cache** is "linear_cache" containing (A_prev, W, b) coming from the forward propagation in the current layer
   

Implement the linear portion of backward propagation for a single layer (layer l)

    Arguments:
    dZ -- Gradient of the cost with respect to the linear output (of current layer l)
    cache -- tuple of values (A_prev, W, b) coming from the forward propagation in the current layer

    Returns:
    dA_prev -- Gradient of the cost with respect to the activation (of the previous layer l-1), same shape as A_prev
    dW -- Gradient of the cost with respect to W (current layer l), same shape as W
    db -- Gradient of the cost with respect to b (current layer l), same shape as b

In [16]:
def linear_backward(dZ, cache):
   
    A_prev, W, b = cache
    m = A_prev.shape[1]

    dW = (1/m) * np.dot(dZ, A_prev.T)
    db = (1/m) * np.sum(dZ, axis=1, keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [17]:
def linear_activation_backward(dA, cache, activation):
    
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = relu_backward(dA, activation_cache)
        
    elif activation == "sigmoid":
        dZ = sigmoid_backward(dA, activation_cache)
    
    dA_prev, dW, db = linear_backward(dZ, linear_cache)
    
    return dA_prev, dW, db

# Obtain Gradient

Implement the backward propagation for the [LINEAR->RELU] * (L-1) -> LINEAR -> SIGMOID group
    
    Arguments:
    AL -- probability vector, output of the forward propagation (L_model_forward())
    Y -- true "label" vector (containing 0 if non-cat, 1 if cat)
    caches -- list of caches containing:
                every cache of linear_activation_forward() with "relu" (it's caches[l], for l in range(L-1) i.e l = 0...L-2)
                the cache of linear_activation_forward() with "sigmoid" (it's caches[L-1])
    
    Returns:
    grads -- A dictionary with the gradients
             grads["dA" + str(l)] = ... 
             grads["dW" + str(l)] = ...
             grads["db" + str(l)] = ... 

In [18]:
def L_model_backward(AL, Y, caches):
   
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    dAL = - (np.divide(Y, AL) - np.divide(1 - Y, 1 - AL))
    
    # Lth layer (SIGMOID -> LINEAR) gradients. Inputs: "dAL, current_cache". Outputs: "grads["dAL-1"], grads["dWL"], grads["dbL"]
    current_cache = caches[L-1] # Last Layer
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, current_cache, "sigmoid")
    
    # Loop from l=L-2 to l=0
    for l in reversed(range(L-1)):
        # lth layer: (RELU -> LINEAR) gradients.
        # Inputs: "grads["dA" + str(l + 1)], current_cache". Outputs: "grads["dA" + str(l)] , grads["dW" + str(l + 1)] , grads["db" + str(l + 1)] 
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, activation = "relu")
        grads["dA" + str(l)] = dA_prev_temp
        grads["dW" + str(l + 1)] = dW_temp
        grads["db" + str(l + 1)] = db_temp

    return grads

# Update Parameters

Update parameters using gradient descent
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    grads -- python dictionary containing your gradients, output of L_model_backward
    
    Returns:
    parameters -- python dictionary containing your updated parameters 
                  parameters["W" + str(l)] = ... 
                  parameters["b" + str(l)] = ...

In [12]:
def update_parameters(parameters, grads, learning_rate):

    L = len(parameters) // 2 # number of layers in the neural network

    # Update rule for each parameter. Use a for loop.
    for l in range(L):
        parameters["W" + str(l+1)] = parameters["W" + str(l+1)] - learning_rate * grads["dW" + str(l+1)]
        parameters["b" + str(l+1)] = parameters["b" + str(l+1)] - learning_rate * grads["db" + str(l+1)]
    return parameters

In [0]:
# parameters, grads = update_parameters_test_case()
# parameters = update_parameters(parameters, grads, 0.1)

# print ("W1 = "+ str(parameters["W1"]))
# print ("b1 = "+ str(parameters["b1"]))
# print ("W2 = "+ str(parameters["W2"]))
# print ("b2 = "+ str(parameters["b2"]))